In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

input_data = pd.read_csv("hw1_input.csv")
output_real = pd.read_csv("hw1_real.csv")
output_imag = pd.read_csv("hw1_img.csv")

combined_data = pd.concat([input_data, output_real, output_imag], axis=1)


scaler = StandardScaler()
combined_data_scaled = scaler.fit_transform(combined_data)


pca = PCA()
pca.fit(combined_data_scaled)


explained_variance = pca.explained_variance_ratio_[:3]
cumulative_variance = explained_variance.cumsum()


explained_variance_df = pd.DataFrame({
    'Principal Component': [f'PC{i+1}' for i in range(3)],
    'Explained Variance Ratio': explained_variance,
    'Cumulative Variance Ratio': cumulative_variance
})

loadings = pca.components_[:3, :11]
loadings_df = pd.DataFrame(loadings, columns=combined_data.columns[:11])
loadings_df.index = [f'PC{i+1}' for i in range(3)]


print("Explained Variance and Cumulative Variance for PC1 to PC3:")
print(explained_variance_df)

print("\nLoadings (Contribution of Each of the First 11 Features to PC1 to PC3):")
print(loadings_df)


Explained Variance and Cumulative Variance for PC1 to PC3:
  Principal Component  Explained Variance Ratio  Cumulative Variance Ratio
0                 PC1                  0.520379                   0.520379
1                 PC2                  0.142717                   0.663096
2                 PC3                  0.110752                   0.773848

Loadings (Contribution of Each of the First 11 Features to PC1 to PC3):
     length of patch  width of patch  height of patch  height of substrate  \
PC1        -0.004894        0.059770         0.004230             0.064432   
PC2        -0.012305       -0.013879        -0.008578            -0.002984   
PC3         0.005707       -0.036499        -0.010962            -0.030202   

     height of solder resist layer  radius of the probe     c_pad  c_antipad  \
PC1                      -0.001097             0.003096 -0.003419  -0.004743   
PC2                      -0.003461             0.028281  0.000656  -0.015860   
PC3            

1-Yes, focusing on the first three components hold nearly the 80% of cumulative variance thus the information, that shows us we can simply explain this data using three components. With just PC1, PC2, and PC3, we capture the most critical design factors. This dimensionality reduction allows us to reduce complexity significantly without losing essential information, we also suggest that these components contain the primary patterns influencing S11 magnitudes.

2-PC1, PC2, and PC3 together explain 77.4% of the total variance, which is close to the 80% threshold we aim to capture.
This means underlying structure represented by these first three components explains that nearly 80% of the variability in the design parameters (and likely much of the impact on S11 magnitude response)   


3.1  Key Takeaways for PC1

Width of patch (0.0598) and height of substrate (0.0644) have the highest positive coefficients in PC1, indicating that they are the strongest contributors to the variance captured by this component.
Dielectric constant of substrate (0.0273) also contributes positively, suggesting that the material property related to substrate plays a role in this main pattern.
The length of patch and height of patch have near-zero loadings, indicating minimal contribution to PC1.
In summary, PC1 is primarily influenced by the width of the patch, height of the substrate, and the dielectric constant of the substrate. These parameters are likely crucial in determining the main geometric and material pattern in the data, which explains the majority of the variance related to S11.

3.2 Key Takeaways for PC2

Radius of the probe (0.0283) and c_probe (0.0262) are the most significant contributors to PC2, indicating that probe positioning and capacitance factors play a prominent role in this secondary pattern.
The width of patch and height of substrate show lower, near-zero contributions in PC2 compared to PC1, suggesting that these geometric parameters are less significant in the variation captured by PC2.
In summary, PC2 is primarily influenced by probe-related parameters such as radius of the probe and c_probe. This component likely represents variation related to the electrical characteristics and positioning of the probe, rather than overall geometry.

3.3 Key Takeaways for PC3
c_probe (0.0373) and radius of the probe (0.0214) are the main contributors to PC3, indicating that probe-related properties are again significant in this component.
Width of patch (-0.0365) and dielectric constant of substrate (-0.0368) have high negative loadings, suggesting that these factors influence the design variability represented in PC3 but in an opposite direction compared to other parameters.
Other parameters such as height of patch and c_antipad have lower loadings, indicating they play a smaller role in this component.

In [ ]:
import pandas as pd

output_real = pd.read_csv("hw1_real.csv")
output_imag = pd.read_csv("hw1_img.csv")

s11_magnitude = (output_real**2 + output_imag**2)**0.5

flattened_magnitudes = s11_magnitude.values.flatten()
flattened_indices = [i % s11_magnitude.shape[1] for i in range(flattened_magnitudes.size)]

magnitude_df = pd.DataFrame({"index": flattened_indices, "magnitude": flattened_magnitudes})
most_critical_frequencies = magnitude_df.sort_values(by="magnitude").drop_duplicates(subset="index").head(10)["index"].tolist()

print("The 10 most critical frequency indices are:", most_critical_frequencies)


The 10 most critical frequency indices are: [71, 28, 27, 126, 155, 85, 88, 55, 26, 172]


3.2.1 The 10 most critical frequency indices are: [71, 28, 27, 126, 155, 85, 88, 55, 26, 172], These will output the indices corresponding to the 10 most critical frequencies where resonance is likely to occur.

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

output_real = pd.read_csv("hw1_real.csv")
output_imag = pd.read_csv("hw1_img.csv")

critical_frequencies = [71, 28, 27, 126, 155, 85, 88, 55, 26, 172]

s11_real_critical = output_real.iloc[:, critical_frequencies]
s11_imag_critical = output_imag.iloc[:, critical_frequencies]
s11_magnitude_critical = (s11_real_critical**2 + s11_imag_critical**2)**0.5

design_params = pd.read_csv("hw1_input.csv")
pca = PCA(n_components=3)
design_params_pca = pca.fit_transform(design_params)

X_train, X_test, y_train, y_test = train_test_split(design_params_pca, s11_magnitude_critical, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

r_squared = model.score(X_test, y_test)
print("R-squared:", r_squared)


Mean Squared Error: 0.018574581957154827
R-squared: 0.5388372915657177


3.2.2 With an R-squared value of approximately 0.54, the model explains about 54% of the variance in S11 magnitudes, leaving 46% unexplained. This suggests that while linear regression captures some primary linear relationships, it fails to account for the more complex, likely nonlinear interactions inherent in electromagnetic behavior, particularly at resonance frequencies. The Mean Squared Error (MSE) of 0.0186 also indicates moderate prediction accuracy, though improvements could be made by incorporating additional principal components or using nonlinear models, which may better capture the intricate dependencies between design parameters and S11 response.

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

output_real = pd.read_csv("hw1_real.csv")
output_imag = pd.read_csv("hw1_img.csv")

individual_frequencies = [71, 28, 27, 126, 155, 85, 88, 55, 26, 172]

s11_real_individual = output_real.iloc[:, individual_frequencies]
s11_imag_individual = output_imag.iloc[:, individual_frequencies]
s11_magnitude_individual = (s11_real_individual**2 + s11_imag_individual**2)**0.5

design_params = pd.read_csv("hw1_input.csv")
pca = PCA(n_components=3)
design_params_pca = pca.fit_transform(design_params)

results = {}

for i, freq in enumerate(individual_frequencies):
    y = s11_magnitude_individual.iloc[:, i]

    X_train, X_test, y_train, y_test = train_test_split(design_params_pca, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r_squared = model.score(X_test, y_test)

    
    results[f"Frequency {freq}"] = {"MSE": mse, "R-squared": r_squared}


for freq, metrics in results.items():
    print(f"{freq}: MSE = {metrics['MSE']}, R-squared = {metrics['R-squared']}")


Frequency 71: MSE = 0.025335397203089357, R-squared = 0.502217629893386
Frequency 28: MSE = 0.013106414407758815, R-squared = 0.49946236197102667
Frequency 27: MSE = 0.012590044810030192, R-squared = 0.49495372024227724
Frequency 126: MSE = 0.021965749181058914, R-squared = 0.5640251070703057
Frequency 155: MSE = 0.024562261044339093, R-squared = 0.4776703895886302
Frequency 85: MSE = 0.014780307675427182, R-squared = 0.6516120764318314
Frequency 88: MSE = 0.014938138949029345, R-squared = 0.6529860047451213
Frequency 55: MSE = 0.01715255889121228, R-squared = 0.6156988843659095
Frequency 26: MSE = 0.012117211894270761, R-squared = 0.48944635524617586
Frequency 172: MSE = 0.029197735515332373, R-squared = 0.440300386102513


3.2.3 R-squared values vary, with some frequencies showing stronger linear relationships with the principal components of the design parameters than others. Frequencies like 85, 88, and 55 have relatively high R-squared values (around 0.65), indicating that these points are more predictable through linear relationships, possibly due to more direct dependencies on the geometric parameters captured by PCA. Conversely, frequencies like 172 and 155 have lower R-squared values (around 0.44-0.48), suggesting more complex or nonlinear interactions that the linear model does not fully capture.

These patterns suggest broader trends in the design space: certain frequency points, possibly near resonance or specific operational bands, are influenced more directly by primary geometric factors, making them easier to predict linearly. In contrast, other frequencies likely involve more intricate dependencies or interactions among design parameters, implying the presence of nonlinear behavior.